In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os 

In [0]:
torch.__version__

'1.4.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
'cropped_Resnet(Raw).ipynb'
 dataset
 dump
 epoch_65.pth
 extract_faceandhand_jorehgaye.ipynb
 fallingDemo.ipynb
 fallingDemo_modified_2.py
 fallingDemo_modified.py
 fallingDemo.py
 loss
 out.jpg
 putTXT_rightplace.ipynb
'Testbeta-Resnet(Raw+Face+Hand).ipynb'
'Testbeta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'testbeta-Resnet(Raw+Face).ipynb'
'Testbeta-Resnet(Raw+Hand).ipynb'
'Testbeta-Resnet(Raw).ipynb'
 TestingModels_first.ipynb
 TestingModels_second.ipynb
 weights


In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

In [0]:
input_path = './dataset/alpha/Choke'

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + '/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + '/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=8),  
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=8)  
}

In [7]:
dataloaders['train'].dataset.classes

['Choking', 'NonChoking']

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2),
               nn.Sigmoid()).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=3):
    new_write = True
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        #f.write('Epoch {}/{}'.format(epoch+1, num_epochs))
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

                  _, preds = torch.max(outputs, 1)
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

                  count = count+1                
                  if (count%40==0):
                    print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if (os.path.isfile('loss/alpha/Choke/raw.txt')):
              new_write=False  
            else :
              print ("No file")
            f = open('loss/alpha/Choke/raw.txt','a+')
            if  (new_write == True ):
              print ("here")
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            else :
              f.write ('\n')
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            
        torch.save(model, 'weights/alpha/Choke/raw/epoch_%d.pth'%(epoch))
        f.close()


    return model

In [14]:

model_trained = train_model(model, criterion, optimizer, num_epochs=100)


Epoch 1/100
----------
40
train loss: 0.6273, acc: 0.6591
No file
here
validation loss: 0.5752, acc: 0.7292
Epoch 2/100
----------
40
train loss: 0.5547, acc: 0.7556
validation loss: 0.5492, acc: 0.7428
Epoch 3/100
----------
40
train loss: 0.5437, acc: 0.7546
validation loss: 0.5230, acc: 0.7834
Epoch 4/100
----------
40
train loss: 0.5215, acc: 0.7843
validation loss: 0.5305, acc: 0.7681
Epoch 5/100
----------
40
train loss: 0.5050, acc: 0.8054
validation loss: 0.5031, acc: 0.8114
Epoch 6/100
----------
40
train loss: 0.4940, acc: 0.8178
validation loss: 0.5234, acc: 0.7726
Epoch 7/100
----------
40
train loss: 0.4929, acc: 0.8165
validation loss: 0.5152, acc: 0.7870
Epoch 8/100
----------
40
train loss: 0.4885, acc: 0.8227
validation loss: 0.5507, acc: 0.7410
Epoch 9/100
----------
40
train loss: 0.4760, acc: 0.8351
validation loss: 0.4798, acc: 0.8339
Epoch 10/100
----------
40
train loss: 0.4758, acc: 0.8348
validation loss: 0.5061, acc: 0.7978
Epoch 11/100
----------
40
train los

In [0]:
!mkdir weights
!mkdir weights/pytorch

In [0]:
torch.save(model_trained.state_dict(), 'weights/pytorch/weightsE3.h5')

In [0]:
# model = models.resnet50(pretrained=False).to(device)
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)
model = (torch.load('weights/pytorch/fallweightsresNet_3_12_201990.pth'))

In [0]:
!ls

darknet		      examples	      opt.py		train_sppe
data		      fallmodel.pth   output		video_ann.py
datalabeler_101.py    fi_data	      PoseFlow		video_demo.py
datalabeler_111.py    fn.py	      pPose_nms.py	webcam_demo.py
datalabeler_249.py    img.png	      __pycache__	weights
dataloader.py	      LICENSE	      README.md		yolo
dataloader_webcam.py  matching.py     requirements.txt
demo.py		      models	      resnetWeights
doc		      online_demo.py  SPPE


In [0]:
import glob
import random
pathFalling = './fi_data/train/Falling/'
pathNonFalling = './fi_data/train/NonFalling/'
accurateFalling = 0
accurateNonFalling = 0

list1= [f for f in glob.glob(pathFalling + "*.jpg", recursive=True)]
list2= [f for f in glob.glob(pathNonFalling + "*.jpg", recursive=True)]
validation_img_pathsFalling=random.sample(list1,100)
validation_img_pathsNonFalling = random.sample(list2,100)
img_listFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsFalling]
img_listNonFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsNonFalling]
print (len(img_listFalling))
print (len(img_listNonFalling))

100
100


In [0]:
validation_batchFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listFalling])
validation_batchNonFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listNonFalling])

In [0]:
pred_logits_tensorFalling = model(validation_batchFalling)
pred_logits_tensorFalling
pred_logits_tensorNonFalling = model(validation_batchNonFalling)
pred_logits_tensorNonFalling

tensor([[-1.5283e+00,  9.0661e-01],
        [-2.2185e+00,  2.1231e+00],
        [-2.7703e+00,  3.2524e+00],
        [-2.4615e+00,  2.0275e+00],
        [-1.1705e+00,  9.6872e-01],
        [-4.7061e-01,  8.8000e-01],
        [-1.1260e+00,  6.5979e-01],
        [-3.8996e+00,  4.1767e+00],
        [-5.8298e-01,  2.0960e-01],
        [-7.5061e-01,  5.5574e-01],
        [-2.2707e+00,  2.1771e+00],
        [-1.5751e+00,  1.3934e+00],
        [-9.6311e-01,  3.2051e-01],
        [-2.7319e+00,  2.2038e+00],
        [-9.1111e-02, -3.2099e-03],
        [ 7.4642e-02, -3.1217e-01],
        [-1.6257e+00,  1.6039e+00],
        [-3.4561e+00,  3.4454e+00],
        [-3.1866e+00,  2.6508e+00],
        [-6.3874e-01,  3.6337e-01],
        [-1.6754e+00,  1.5005e+00],
        [-2.6561e+00,  2.5272e+00],
        [-4.2902e+00,  3.4453e+00],
        [-5.9180e+00,  5.5812e+00],
        [-1.1807e+00,  3.4439e-01],
        [ 5.9854e-01, -1.1552e+00],
        [-1.0886e+00,  1.4284e+00],
        [-1.3321e+00,  6.760

In [0]:
pred_probsFalling = F.softmax(pred_logits_tensorFalling, dim=1).cpu().data.numpy()
print (pred_probsFalling)
pred_probsNonFalling = F.softmax(pred_logits_tensorNonFalling, dim=1).cpu().data.numpy()
print (pred_probsNonFalling)

[[0.44844374 0.5515562 ]
 [0.12725133 0.8727487 ]
 [0.09804448 0.9019555 ]
 [0.00425655 0.99574345]
 [0.8194356  0.18056442]
 [0.24643923 0.7535607 ]
 [0.13835269 0.8616473 ]
 [0.1574074  0.84259254]
 [0.8485861  0.1514139 ]
 [0.32346088 0.6765391 ]
 [0.41357878 0.58642125]
 [0.15117285 0.8488271 ]
 [0.6003375  0.39966255]
 [0.02290936 0.9770906 ]
 [0.10023551 0.89976454]
 [0.29872367 0.70127636]
 [0.04371396 0.956286  ]
 [0.61934406 0.38065597]
 [0.30597326 0.6940267 ]
 [0.16753031 0.8324697 ]
 [0.66433    0.33567002]
 [0.84191    0.15808995]
 [0.6628018  0.33719823]
 [0.4977156  0.50228435]
 [0.87176895 0.12823105]
 [0.02149463 0.9785054 ]
 [0.48655316 0.5134468 ]
 [0.44041517 0.55958486]
 [0.1114507  0.88854927]
 [0.5616096  0.4383904 ]
 [0.570884   0.42911607]
 [0.32838956 0.6716105 ]
 [0.42478982 0.5752102 ]
 [0.5371839  0.46281612]
 [0.02694354 0.9730565 ]
 [0.61242115 0.3875789 ]
 [0.21000993 0.7899901 ]
 [0.32981074 0.6701893 ]
 [0.23768397 0.76231605]
 [0.846486   0.15351407]


In [0]:
i=0
accurateFalling=0
for i in range(len(pred_probsFalling)):
  if (pred_probsFalling[i][0]> 0.5):
    accurateFalling= accurateFalling +1 
i=0
accurateNonFalling=0
for i in range(len(pred_probsNonFalling)):
  if (pred_probsNonFalling[i][1]> 0.5):
    accurateNonFalling= accurateNonFalling +1 


In [0]:
overallAccuracy = (accurateNonFalling+accurateFalling)/(len(img_listFalling)+len(img_listNonFalling))

In [0]:
print (accurateFalling)
print (accurateNonFalling)

41
92


In [0]:
print (overallAccuracy)

0.665


In [0]:
ig, axs = plt.subplots(1, len(img_listFalling), figsize=(150, 150))
for i, img in enumerate(img_listFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsFalling[i,0],
                                                          100*pred_probsFalling[i,1]))
    ax.imshow(img)

In [0]:
ig, axs = plt.subplots(1, len(img_listNonFalling), figsize=(150, 150))
for i, img in enumerate(img_listNonFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsNonFalling[i,0],
                                                          100*pred_probsNonFalling[i,1]))
    ax.imshow(img)